In [10]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
from scipy.stats import norm
from pandas_profiling import ProfileReport

from datetime import datetime

import glob
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
from plotly.subplots import make_subplots
import plotly.express as px

plt.style.use("bmh")
plt.rcParams['font.family'] = 'DejaVu Sans'

In [39]:
from seaborn import heatmap

In [15]:
from numpy import inf

Подгружаю таблицу с данными о времени новости и к какому тикеру она относится. Я продублировала время в привычном виде, удалила столбцы, которыми не буду пользоваться.

In [2]:
ticker_price = pd.read_hdf("./ticker_price_moment_exit.hd5")
ticker_price['newtime'] = pd.to_datetime(ticker_price['timestamp'], unit='s') 
news_moment = ticker_price[['news_id', 'ticker','timestamp','newtime', 'current_price']]
news_moment

,news_id,ticker,timestamp,newtime,current_price
0,0000d467d1a52a6a84fc8b30adbe8ec2,VS,1629716406,2021-08-23 11:00:06,4.24
1,0000d467d1a52a6a84fc8b30adbe8ec2,VSSYW,1629716406,2021-08-23 11:00:06,2.08
3,00091f238ca72bd41d8baf14eb089f51,BBBY,1631707209,2021-09-15 12:00:09,23.83
4,000de19c850fb98b434e5adf871f6618,NATR,1630504816,2021-09-01 14:00:16,17.33
5,0012e7727394e60ef02ab76b3bf62a80,GM,1629939723,2021-08-26 01:02:03,49.7
...,...,...,...,...,...
29822,ffeaf7d8b6755c4304c663aaeb151a86,SHG,1631108656,2021-09-08 13:44:16,33.1
29823,ffecc02832a37b16899cbe94c55ce324,CMCO,1631538007,2021-09-13 13:00:07,43.68
29824,ffed9b091723da2ce615a04106f0fd24,FOX,1631205010,2021-09-09 16:30:10,33.61
29825,ffed9b091723da2ce615a04106f0fd24,FOXA,1631205010,2021-09-09 16:30:10,36.295


Подгружаю таблицу со скорами и оставляю только те, новости, которые модель посчитала значимыми.

In [4]:
scores_retrotesting = pd.read_hdf("./scores_retrotesting.hd5")
scores = scores_retrotesting[scores_retrotesting['final_probabilities'] > 0.9][['news_id', 'ticker']]
scores

,news_id,ticker
77,1256a14eb635a4d5ce7265db89b17e1a,GP
111,1aaa42da4e6d7415f53b737169f2fc1c,AGFY
117,1b754b1268deb3b6b6ed6a83ee7104ac,HALL
121,1be2400fa45b7db74aaf05b129714941,EVOK
203,2ea8671cbff9e657e2462e010a644af1,IPWR
...,...,...
24063,59b2c7d195a1f772649f6afd250bbeef,ONTX
24093,6448224e359a308d5a12817e369edd98,AIM
24266,d6c11e3c23217f0414ceae4ce89f23e4,CELU
24267,d6c11e3c23217f0414ceae4ce89f23e4,ONCT


Объединяю две таблицы вместе для удобства добывания информации.

In [18]:
alldata = scores.merge(news_moment, how ='left', on = ['news_id', 'ticker']).dropna()
alldata

,news_id,ticker,timestamp,newtime,current_price
86,26aae81ebc585014e2c54648d5ab09d5,CBAT,1629720014,2021-08-23 12:00:14,3.0
87,2916f7b2d66812594ca2301bf8aa40e3,PANW,1629749116,2021-08-23 20:05:16,372.63
88,4ba4d036649e90396f3af6b7615f8f72,MARK,1629749710,2021-08-23 20:15:10,1.14
89,82f80a9ce0349dae876e7d4e31f346b8,AXSM,1629712806,2021-08-23 10:00:06,21.56
90,831f3786381279a64c7243fce8002839,BBIG,1629750631,2021-08-23 20:30:31,2.46
...,...,...,...,...,...
195,59b2c7d195a1f772649f6afd250bbeef,ONTX,1632139310,2021-09-20 12:01:50,5.11
196,6448224e359a308d5a12817e369edd98,AIM,1632143705,2021-09-20 13:15:05,2.04
197,d6c11e3c23217f0414ceae4ce89f23e4,CELU,1632139309,2021-09-20 12:01:49,6.24
198,d6c11e3c23217f0414ceae4ce89f23e4,ONCT,1632139309,2021-09-20 12:01:49,4.24


Смотрим на поминутную стоимость акций. (Все дни склеены в ону таблицу)

In [7]:
all_files = glob.glob("./new/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col = None, header=0)
    li.append(df)

alldays = pd.concat(li, ignore_index = True)
alldays['newtime'] = pd.to_datetime(alldays['time'], unit='s')
intraday_forall = alldays[['ticker','time', 'newtime', 'close']]
intraday_forall = intraday_forall.sort_values(by = ['time'])
intraday_forall

,ticker,time,newtime,close
3292355,FCX,1628755200,2021-08-12 08:00:00,38.7600
3579202,AHPI,1628755200,2021-08-12 08:00:00,12.9000
3337449,TME,1628755200,2021-08-12 08:00:00,10.5000
3186471,QQQ,1628755200,2021-08-12 08:00:00,365.6000
3173045,BBI,1628755200,2021-08-12 08:00:00,0.6670
...,...,...,...,...
726923,UPST,1632144900,2021-09-20 13:35:00,291.5100
727270,UBER,1632144900,2021-09-20 13:35:00,39.0700
734651,NOC,1632144900,2021-09-20 13:35:00,347.1300
727720,FOLD,1632144900,2021-09-20 13:35:00,10.9000


Основная часть:

In [19]:
def best_profit(ticker, money = 100, data2 = alldata): #для каждого тикера будем находить такие прибыль, х и у, 
                                                        #что х -- это пройденные минуты после новости, 
                                                        #у -- после открытия позиции, и прибыль максимальна
    profit = float(-inf) #прибыль
    positions = 1
    time_in = 0 #пройденные минуты после новости
    time_out = 0 #пройденные минуты после открытия позиции
    
    newsdate = data2[data2['ticker'] == ticker]['timestamp'].values[0] #находим дату новости по тикеру 
                                                                       #(считаем, что у каждого тикера одна новостьб вроде так и есть)
    

    for t1 in range(6): #0-5 минут после новости
        date_in = newsdate - newsdate%60 + t1*60 #отбрасываем секунды и находим время открытия позиции
        name = datetime.fromtimestamp(date_in).strftime("%Y-%m-%d")
        data1 = pd.read_csv('./new/new-' + name + '.csv')
        
        if not data1[(data1['time'] == date_in) & (data1['ticker'] == ticker)].empty: #в данных не для каждого тикера указана цена в конкретную минуту
        #заходим в позицию: в длинную или короткую зависит от новости
            positions = money/data1[(data1['time'] == date_in) & (data1['ticker'] == ticker)]['close'].values[0]
            price1 = data1[(data1['time'] == date_in) & (data1['ticker'] == ticker)]['close'].values[0]
        
            for t2 in [1,2,5,10,15]: #1-15 минут после открытия позиции
                date_out = date_in + t2*60 #время выхода из позиции
            
                if not data1[(data1['time'] == date_out) & (data1['ticker'] == ticker)].empty:
                    price2 = data1[(data1['time'] == date_out) & (data1['ticker'] == ticker)]['close'].values[0]
                    #доход с одной позиции
                    profit1 = price2 - price1
                    if profit1 > profit:
                        profit, time_in, time_out = profit1, t1, t2
    
    return(profit*positions, time_in, time_out)

In [ ]:
best_profit_series = []
t1_series = []
t2_series = []
result = alldata['ticker'].apply(best_profit)

In [81]:
for a in result.values:
    best_profit_series.append(a[0])
    t1_series.append(a[1])
    t2_series.append(a[2])

В таблицу alldata добавляем данные о наибольшей прибыли, которую мы могли бы получить, если бы зашли через open_pos_time минут после новости, и вышли бы еще через close_pos_time

In [82]:
alldata['best_profit'] = best_profit_series
alldata['open_pos_time'] = t1_series
alldata['close_pos_time'] = t2_series

alldata = alldata.sort_values(by = ['best_profit'])
alldata

,news_id,ticker,timestamp,newtime,current_price,best_profit,open_pos_time,close_pos_time
29,f4c7d7e7ce70bcfe0126a2425cafc5c4,RLGT,1630361047,2021-08-30 22:04:07,6.61,-inf,0,0
46,4bd93dc51354981f6c92033ea7ecae74,DADA,1631052004,2021-09-07 22:00:04,26.41,-inf,0,0
96,8cd4c1473c0d7ba9f1371e1e522d6290,JWEL,1631566805,2021-09-13 21:00:05,5.76,-inf,0,0
100,6c0421adaac84bed2ae7d2710772fef0,PNT,1631619187,2021-09-14 11:33:07,9.15,0.000000,0,1
102,0bd770b807058a711f624c671a787733,MGTA,1631707214,2021-09-15 12:00:14,6.13,0.000000,0,1
...,...,...,...,...,...,...,...,...
101,a12d94e4122e67897920e86052a1821e,AXSM,1631617207,2021-09-14 11:00:07,27.48,13.521739,3,15
6,9417952eff172fae2bc1d114a15b3955,TBPH,1629749115,2021-08-23 20:05:15,14.17,17.800000,5,15
103,468779d113f1b8b47d0819be832b2e9b,TBPH,1631700318,2021-09-15 10:05:18,8.04,17.800000,5,15
66,9a3016a2eddd4c836dea4fd09c5e410b,PRQR,1631132103,2021-09-08 20:15:03,6.81,20.851689,0,15


Построим гистограму, чтобы увидеть на каких тикерах была получена лучшая доходность в первые же минуты.

In [84]:
x = alldata['ticker']
y = alldata['best_profit']

fig = go.Figure(data=[go.Bar(x = x, y = y)])

fig.update_traces(marker_color='blue', marker_line_color='black',
                  marker_line_width=1.8, opacity=0.9)
fig.show()

 Очень сильно отрывается сектор биофармы. У BLU максимальная доходность среди всех, причем такой скачок был именно в первую минуту. (Там минуты 0 и 1)

In [31]:
sample = intraday_forall[intraday_forall['ticker'] == 'BLU']
sample

,ticker,time,newtime,close
4093633,BLU,1629721800,2021-08-23 12:30:00,3.1099
4093634,BLU,1629721860,2021-08-23 12:31:00,3.1099
4093635,BLU,1629721920,2021-08-23 12:32:00,3.1099
4093636,BLU,1629721980,2021-08-23 12:33:00,3.1099
4093637,BLU,1629722040,2021-08-23 12:34:00,3.1099
...,...,...,...,...
9720156,BLU,1631663700,2021-09-14 23:55:00,5.6000
9720157,BLU,1631663760,2021-09-14 23:56:00,5.6000
9720158,BLU,1631663820,2021-09-14 23:57:00,5.6500
9720159,BLU,1631663880,2021-09-14 23:58:00,5.7000


Посмотрим на поведение цены акции.

In [32]:
fig = go.Figure(data=go.Scatter(x = sample['newtime'], y = sample['close']))
fig.show()

# Продолжение

In [21]:
def profit(data,newstime,t1,t2):
    time_in = newstime - newstime%60 + t1*60
    price1 = data[data["time"] == time_in]
    
    if( price1.empty):
        return 0
    
    positions = 100/price1['close'].values[0]
    time_out = time_in + t2*60
    price2 = data[data["time"] == time_out]
    
    if(price2.empty):
        return 0

    return (price2['close'].values[0] - price1['close'].values[0])*positions

In [22]:
array_forall  = np.zeros((6,6))

t1  = [0,1,2,3,4,5] 
t2 = [1,2,5,10,15,20] 

In [23]:
for a in alldata['news_id']:
    newstime = alldata[alldata['news_id'] == a]['timestamp'].values[0]
    ticker = alldata[alldata['news_id'] == a]['ticker'].values[0]
    prices = intraday_forall[intraday_forall['ticker'] == ticker]
    if not prices.empty:
        for i in t1:
            for j in t2:
                array_forall[t1[i]][t2.index(j)] += profit(prices, newstime, i, j)

In [28]:
from  matplotlib.colors import LinearSegmentedColormap
c = ["darkred","red","lightcoral","white", "palegreen","green","darkgreen"]
v = [0,.15,.4,.5,0.6,.9,1.]
l = list(zip(v,c))
cmap1 = LinearSegmentedColormap.from_list('rg',l, N=256)

In [48]:
fig = plt.figure(figsize=(10, 8))

ax = sns.heatmap(array_forall, center = 0, cmap = cmap1, annot=True)

plt.show()

In [30]:
maximum = np.amax(array_forall)
maximum

20.94794089223957

In [31]:
np.where(maximum == array_forall)

(array([0]), array([0]))

In [32]:
industries = pd.read_csv('article_metadata.csv')
industries.nunique()

Unnamed: 0     33736
title          33174
description    30816
source.name       22
symbols        11251
industries      1974
sectors          392
time zone          1
news_id        33736
dtype: int64

In [33]:
industries.head()

,Unnamed: 0,title,description,source.name,symbols,industries,sectors,time zone,news_id
0,0,Salem Podcast Network Adds Jenna Ellis to the ...,"Salem Media Group, Inc. (NASDAQ:SALM) announce...",BusinessWire,['SALM'],['Broadcasting - Radio'],['Consumer Cyclical'],UTC 0,000fb4a135119a9cf49fe51f91e37add
1,1,"PureCycle, SK Global Chemical Partner To Open ...",PureCycle Technologies Inc (NASDAQ: PCT) signe...,Benzinga,['PCT'],[],[],UTC 0,000fda8be4442b6ffdcc247693075ffa
2,2,Rackspace shares slide as execution challenges...,da-kuk/E+ via Getty Images Waiting for executi...,Seeking Alpha,['RXT'],[],[],UTC 0,00183e71753b66ed27eb5a59886eed7d
3,3,"Burlington Stores, Inc. Announces Second Quart...","BURLINGTON, N.J., Aug. 12, 2021 (GLOBE NEWSWIR...",GlobeNewswire,['BURL'],['Discount Stores'],['Consumer Defensive'],UTC 0,0041788faa1ef738950595f0a7fb2668
4,4,Shutterstock Announces Integration With OpenTe...,"NEW YORK, Aug. 12, 2021 /PRNewswire/ -- Shutte...",PR Newswire,['SSTK'],['Internet Content & Information'],['Technology'],UTC 0,005e815d243c2cb606e8d23cbf064255


In [34]:
industries = industries[industries['industries'] != '[]']
industries.head()

,Unnamed: 0,title,description,source.name,symbols,industries,sectors,time zone,news_id
0,0,Salem Podcast Network Adds Jenna Ellis to the ...,"Salem Media Group, Inc. (NASDAQ:SALM) announce...",BusinessWire,['SALM'],['Broadcasting - Radio'],['Consumer Cyclical'],UTC 0,000fb4a135119a9cf49fe51f91e37add
3,3,"Burlington Stores, Inc. Announces Second Quart...","BURLINGTON, N.J., Aug. 12, 2021 (GLOBE NEWSWIR...",GlobeNewswire,['BURL'],['Discount Stores'],['Consumer Defensive'],UTC 0,0041788faa1ef738950595f0a7fb2668
4,4,Shutterstock Announces Integration With OpenTe...,"NEW YORK, Aug. 12, 2021 /PRNewswire/ -- Shutte...",PR Newswire,['SSTK'],['Internet Content & Information'],['Technology'],UTC 0,005e815d243c2cb606e8d23cbf064255
6,6,Alithya reports record first quarter fiscal 20...,"Industry leading organic growth and bookings, ...",PR Newswire,['ALYA'],['Information Technology Services'],['Technology'],UTC 0,008cc24b6c7b3411c1de52d6ab579c6c
7,7,Douglas Dynamics Is Not Cheap Enough Yet,NaN,Seeking Alpha Stock Ideas,['PLOW'],['Auto Parts'],['Consumer Cyclical'],UTC 0,009f17a0d5247c3e5eb52dcc5a9fc037


In [35]:
internet = industries[industries['industries'] == '[\'Internet Content & Information\']'].merge(alldata, how ='inner', on = ['news_id'])
internet.head()

,Unnamed: 0,title,description,source.name,symbols,industries,sectors,time zone,news_id,ticker,timestamp,newtime,current_price
0,9757,Remark Holdings Announces Fiscal Second Quarte...,"LAS VEGAS, Aug. 23, 2021 /PRNewswire/ -- Remar...",PR Newswire,['MARK'],['Internet Content & Information'],['Technology'],UTC 0,4ba4d036649e90396f3af6b7615f8f72,MARK,1629749710,2021-08-23 20:15:10,1.14
1,23391,Luokung Announces eMapGo to Provide Autonomous...,"BEIJING, Sept. 7, 2021 /PRNewswire/ -- Luokung...",PR Newswire,['LKCO'],['Internet Content & Information'],['Technology'],UTC 0,97d75edae27fc4569159e87ac8e1d9fc,LKCO,1631019633,2021-09-07 13:00:33,1.52
2,23801,Qutoutiao Inc. Reports Second Quarter 2021 Una...,"SHANGHAI, China, Sept. 07, 2021 (GLOBE NEWSWIR...",GlobeNewswire,['QTT'],['Internet Content & Information'],['Technology'],UTC 0,d9b6a09f96c00b0c16036fc048ee76b0,QTT,1631012414,2021-09-07 11:00:14,1.42
3,26683,JOYY Inc. Announces New Share Repurchase Plan,"SINGAPORE, Sept. 09, 2021 (GLOBE NEWSWIRE) -- ...",GlobeNewswire,['YY'],['Internet Content & Information'],['Technology'],UTC 0,84b41740a762ce03b8570384a3f16634,YY,1631185208,2021-09-09 11:00:08,64.56


In [36]:
array_internet  = np.zeros((6,6))
for a in internet['news_id']:
    newstime = internet[internet['news_id'] == a]['timestamp'].values[0]
    ticker = internet[internet['news_id'] == a]['ticker'].values[0]
    prices = intraday_forall[intraday_forall['ticker'] == ticker]
    if not prices.empty:
        for i in t1:
            for j in t2:
                array_internet[t1[i]][t2.index(j)] += profit(prices, newstime, i, j)

In [37]:
fig = plt.figure(figsize=(10, 8))

ax = sns.heatmap(array_internet, center = 0, cmap = cmap1, annot=True)

plt.show()

In [134]:
np.amax(array_internet)

2.981847110725369

In [135]:
np.where(array_internet == np.amax(array_internet))

(array([2, 3]), array([3, 3]))

In [144]:
def ind_slice (data):
    array_data = np.zeros((6,6))
    for a in data['news_id']:
        newstime = data[data['news_id'] == a]['timestamp'].values[0]
        ticker = data[data['news_id'] == a]['ticker'].values[0]
        prices = intraday_forall[intraday_forall['ticker'] == ticker]
        if not prices.empty:
            for i in t1:
                for j in t2:
                    array_data[t1[i]][t2.index(j)] += profit(prices, newstime, i, j)
            print(data['industries'].values[0], 'best profit = ',np.amax(array_data),'x = ',np.where(array_data == np.amax(array_data))[0][0], 'y = ',np.where(array_data == np.amax(array_data))[1][0])

In [151]:
for a in set(industries['industries']):
    ind_data = industries[industries['industries'] == a].merge(alldata, how ='inner', on = ['news_id'])
    ind_slice(ind_data)

['Pollution & Treatment Controls', 'Other'] best profit =  0.0 x =  0 y =  0
['Contract Manufacturers'] best profit =  9.821428571428573 x =  3 y =  5
['Capital Markets'] best profit =  3.9387308533916783 x =  3 y =  5
['Electronics Distribution'] best profit =  0.0 x =  0 y =  0
['Telecom Services'] best profit =  3.3088785746369123 x =  1 y =  1
['Electronic Components'] best profit =  1.6339869281045691 x =  5 y =  3
['Electronic Components'] best profit =  4.411764705882349 x =  5 y =  5
['Solar'] best profit =  1.1826821541710562 x =  5 y =  5
['Diversified Industrials'] best profit =  0.0 x =  0 y =  0
['Diversified Industrials'] best profit =  0.005102301137812591 x =  5 y =  0
['Information Technology Services'] best profit =  0.0 x =  0 y =  0
['Communication Equipment'] best profit =  0.0 x =  0 y =  0
['Communication Equipment'] best profit =  2.003088468291014 x =  0 y =  4
['Insurance - Property & Casualty'] best profit =  0.0 x =  0 y =  0
['Packaged Foods'] best profit =

KeyboardInterrupt: 

In [147]:
for a in set(industries['industries']):
    print(a)

['Software - Infrastructure', 'Software - Application', 'Household & Personal Products']
['Utilities - Diversified', 'Business Services']
['Drug Manufacturers - Major', 'Auto Parts']
['Biotechnology', 'Telecom Services', 'Software - Infrastructure']
['Discount Stores', 'Home Improvement Stores']
['REIT - Office', 'Semiconductors', 'Software - Infrastructure', 'Residential Construction']
['Specialty Retail', 'Packaged Foods']
['Banks - Global', 'Telecom Services', 'Consumer Electronics']
['Discount Stores', 'Media - Diversified']
['Footwear & Accessories', 'Telecom Services']
['Banks - Regional - Europe', 'REIT - Diversified']
['Pollution & Treatment Controls', 'Other']
['Specialty Retail', 'Software - Application', 'Discount Stores', 'Banks - Global']
['Specialty Chemicals', 'Industrial Metals & Minerals', 'Chemicals']
['Business Services', 'Discount Stores']
['Leisure', 'Electronic Gaming & Multimedia']
['Biotechnology', 'Specialty Retail']
['Telecom Services', 'Packaged Foods', 'Elec

['Pharmaceutical Retailers', 'Restaurants', 'Discount Stores']
['Footwear & Accessories', 'Apparel Manufacturing', 'Apparel Stores']
['Diagnostics & Research', 'Communication Equipment']
['Specialty Retail', 'Auto Manufacturers', 'Semiconductors', 'Drug Manufacturers - Major', 'Discount Stores']
['Specialty Retail', 'Media - Diversified']
['Chemicals']
['Capital Markets', 'Business Services']
['REIT - Residential', 'Packaged Foods', 'Residential Construction', 'Biotechnology']
['Broadcasting - TV']
['Asset Management', 'Banks - Global', 'Credit Services']
['Apparel Stores', 'Credit Services']
['Residential Construction']
['Capital Markets', 'Semiconductor Equipment & Materials', 'Residential Construction']
['Consumer Electronics', 'Internet Content & Information', 'Asset Management']
['Asset Management', 'Industrial Distribution']
['Biotechnology', 'Electronic Gaming & Multimedia']
['Conglomerates']
['Credit Services', 'Internet Content & Information']
['Education & Training Services',